In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import duckdb
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date() - timedelta(days=1))
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")

Today's date: 2025-11-18


In [2]:
%run ./common_utils.ipynb

In [4]:
def create_actuals_tbl(con):
    df_lines = pd.read_csv(f"../tables/{year}/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[(df_lines.Date == now)]
    df_log = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df = con.execute("""SELECT t2.*, t1.* EXCLUDE(Date, Team, Player, Pos) FROM df_log t1 JOIN df_lines t2 
                        ON t1.Team = t2.Team AND t1.Player = t2.Player""").fetchdf()
    cols = []
    for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
        df[f'{stat}_Result'] = np.where(df[stat] > df[f'{stat}_line'], 'O', 'U')
        df[f'{stat}_Result'] = np.where(df[f'{stat}_line'].isnull(), np.nan, df[f'{stat}_Result'])
        df[f'{stat}_Diff'] = df[stat] - df[f'{stat}_line']
        cols.append(stat)
        cols.append(f'{stat}_line')
        cols.append(f'{stat}_Diff')
        cols.append(f'{stat}_Result')
    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    
    df_stats = pd.read_csv(f"../tables/{year}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, df.Player, df.Pos, MP, 
                            df_stats.* EXCLUDE(Date, Team, Player, {lines}, {diffs}), 
                            df.* EXCLUDE(game_id, Date, Team, Player, Pos, MP) FROM df JOIN df_stats 
                           ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                           """).fetchdf()
#     partition_save_df(df_actuals, f"../tables/{year}/parlay_actuals.csv")
    display(df_actuals)

    return df_actuals
                      
df = create_actuals_tbl(con)

,game_id,Date,Team,Player,Pos,MP,B2B,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,Off_AST,AVG_AST_H2H,Def_AST,Off_REB,AVG_REB_H2H,Def_REB,Off_PR,AVG_PR_H2H,Def_PR,Off_PA,AVG_PA_H2H,Def_PA,Off_RA,AVG_RA_H2H,Def_RA,Off_PRA,AVG_PRA_H2H,Def_PRA,Off_TPM,AVG_TPM_H2H,Def_TPM,TPM_line,TPM_Diff,Off_STL,AVG_STL_H2H,Def_STL,STL_line,STL_Diff,Off_BLK,AVG_BLK_H2H,Def_BLK,BLK_line,BLK_Diff,Off_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,STL_BLK_line,STL_BLK_Diff,PTS,PTS_line,PTS_Diff,PTS_Result,AST,AST_line,AST_Diff,AST_Result,REB,REB_line,REB_Diff,REB_Result,PR,PR_line,PR_Diff,PR_Result,PA,PA_line,PA_Diff,PA_Result,RA,RA_line,RA_Diff,RA_Result,PRA,PRA_line,PRA_Diff,PRA_Result


In [48]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, AVG_{stat}_H2H, Def_{stat}, {stat}_line, {stat}, {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL
                            ORDER BY {stat}_Diff DESC
                            LIMIT 15""").fetchdf())

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251118_UTA_LAL,2025-11-18,UTA,0,Keyonte George,PG,35.35,LAL,22.733333,17.250000,17.230769,19.5,34,14.5,O
1,20251118_GSW_ORL,2025-11-18,GSW,0,Jimmy Butler,SF,38.23,ORL,20.066667,11.285714,19.800000,19.5,33,13.5,O
2,20251118_GSW_ORL,2025-11-18,ORL,0,Anthony Black,PG,32.75,GSW,11.733333,6.400000,20.611111,10.5,21,10.5,O
3,20251118_BOS_BRK,2025-11-18,BRK,0,Day'Ron Sharpe,C,17.90,BOS,7.153846,9.000000,17.500000,5.5,16,10.5,O
4,20251118_UTA_LAL,2025-11-18,LAL,0,Jake LaRavia,PF,25.50,UTA,10.600000,5.000000,18.100000,6.5,16,9.5,O
5,20251118_MEM_SAS,2025-11-18,SAS,0,Harrison Barnes,PF,32.15,MEM,12.857143,11.125000,18.000000,13.5,23,9.5,O
6,20251118_MEM_SAS,2025-11-18,MEM,0,Cedric Coward,SG,25.50,SAS,14.333333,NaN,16.363636,11.5,19,7.5,O
7,20251118_GSW_ORL,2025-11-18,GSW,0,Stephen Curry,PG,34.22,ORL,27.923077,33.400000,22.384615,26.5,34,7.5,O
8,20251118_DET_ATL,2025-11-18,ATL,1,Nickeil Alexander-Walker,SG,36.72,DET,18.142857,12.000000,15.562500,17.5,24,6.5,O
9,20251118_BOS_BRK,2025-11-18,BOS,0,Payton Pritchard,PG,35.25,BRK,16.733333,15.666667,21.500000,15.5,22,6.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,AVG_AST_H2H,Def_AST,AST_line,AST,AST_Diff,AST_Result
0,20251118_UTA_LAL,2025-11-18,LAL,0,LeBron James,SF,29.62,UTA,12.000000,10.333333,5.600000,4.5,12,7.5,O
1,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,3.777778,2.750000,4.272727,3.5,9,5.5,O
2,20251118_GSW_ORL,2025-11-18,GSW,0,Stephen Curry,PG,34.22,ORL,4.416667,6.600000,7.153846,4.5,9,4.5,O
3,20251118_BOS_BRK,2025-11-18,BRK,0,Terance Mann,SG,27.22,BOS,4.000000,4.400000,4.538462,2.5,7,4.5,O
4,20251118_GSW_ORL,2025-11-18,ORL,0,Jalen Suggs,PG,31.93,GSW,4.727273,NaN,7.500000,3.5,8,4.5,O
5,20251118_DET_ATL,2025-11-18,ATL,1,Jalen Johnson,SF,39.10,DET,7.071429,5.200000,2.727273,5.5,9,3.5,O
6,20251118_BOS_BRK,2025-11-18,BRK,0,Ziaire Williams,SF,27.33,BOS,1.600000,1.833333,3.076923,0.5,3,2.5,O
7,20251118_PHO_POR,2025-11-18,PHO,0,Dillon Brooks,SF,25.63,POR,2.714286,1.875000,3.583333,1.5,4,2.5,O
8,20251118_PHO_POR,2025-11-18,PHO,0,Ryan Dunn,SF,26.53,POR,2.230769,1.400000,3.583333,1.5,4,2.5,O
9,20251118_UTA_LAL,2025-11-18,UTA,0,Jusuf Nurkic,C,28.87,LAL,4.066667,4.300000,3.000000,3.5,6,2.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_REB,AVG_REB_H2H,Def_REB,REB_line,REB,REB_Diff,REB_Result
0,20251118_BOS_BRK,2025-11-18,BOS,0,Payton Pritchard,PG,35.25,BRK,4.333333,4.333333,5.000000,3.5,10,6.5,O
1,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,4.700000,3.500000,3.909091,3.5,9,5.5,O
2,20251118_UTA_LAL,2025-11-18,LAL,0,Deandre Ayton,C,30.00,UTA,9.461538,7.500000,10.692308,8.5,14,5.5,O
3,20251118_MEM_SAS,2025-11-18,MEM,0,Cedric Coward,SG,25.50,SAS,6.400000,NaN,3.909091,5.5,11,5.5,O
4,20251118_GSW_ORL,2025-11-18,ORL,0,Wendell Carter Jr.,C,35.12,GSW,7.666667,8.600000,10.300000,8.5,12,3.5,O
5,20251118_PHO_POR,2025-11-18,POR,0,Donovan Clingan,C,23.45,PHO,9.214286,6.800000,10.375000,9.5,12,2.5,O
6,20251118_PHO_POR,2025-11-18,PHO,0,Devin Booker,SG,29.05,POR,4.500000,5.000000,3.833333,3.5,6,2.5,O
7,20251118_MEM_SAS,2025-11-18,MEM,0,Jock Landale,C,17.55,SAS,5.533333,3.750000,8.857143,4.5,7,2.5,O
8,20251118_MEM_SAS,2025-11-18,MEM,0,Zach Edey,C,25.08,SAS,9.000000,8.600000,8.857143,8.5,11,2.5,O
9,20251118_GSW_ORL,2025-11-18,ORL,0,Desmond Bane,SG,39.17,GSW,4.714286,4.428571,3.285714,4.5,6,1.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PR,AVG_PR_H2H,Def_PR,PR_line,PR,PR_Diff,PR_Result
0,20251118_GSW_ORL,2025-11-18,GSW,0,Jimmy Butler,SF,38.23,ORL,25.533333,15.142857,26.066667,25.5,40,14.5,O
1,20251118_UTA_LAL,2025-11-18,UTA,0,Keyonte George,PG,35.35,LAL,26.800000,20.875000,20.384615,23.5,38,14.5,O
2,20251118_MEM_SAS,2025-11-18,MEM,0,Cedric Coward,SG,25.50,SAS,20.733333,NaN,20.272727,17.5,30,12.5,O
3,20251118_BOS_BRK,2025-11-18,BRK,0,Day'Ron Sharpe,C,17.90,BOS,12.538462,15.714286,28.000000,10.5,23,12.5,O
4,20251118_BOS_BRK,2025-11-18,BOS,0,Payton Pritchard,PG,35.25,BRK,21.066667,20.000000,26.500000,20.5,32,11.5,O
5,20251118_MEM_SAS,2025-11-18,SAS,0,Harrison Barnes,PF,32.15,MEM,15.785714,14.750000,25.083333,17.5,28,10.5,O
6,20251118_GSW_ORL,2025-11-18,ORL,0,Anthony Black,PG,32.75,GSW,14.800000,8.000000,26.222222,14.5,25,10.5,O
7,20251118_UTA_LAL,2025-11-18,LAL,0,Deandre Ayton,C,30.00,UTA,25.285714,18.750000,28.153846,23.5,34,10.5,O
8,20251118_GSW_ORL,2025-11-18,ORL,0,Wendell Carter Jr.,C,35.12,GSW,20.400000,17.600000,29.500000,20.5,29,8.5,O
9,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,13.300000,9.000000,20.272727,14.5,23,8.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,AVG_PA_H2H,Def_PA,PA_line,PA,PA_Diff,PA_Result
0,20251118_UTA_LAL,2025-11-18,UTA,0,Keyonte George,PG,35.35,LAL,29.800000,23.250000,23.846154,25.5,42,16.5,O
1,20251118_GSW_ORL,2025-11-18,GSW,0,Jimmy Butler,SF,38.23,ORL,24.800000,15.000000,23.200000,24.5,37,12.5,O
2,20251118_GSW_ORL,2025-11-18,GSW,0,Stephen Curry,PG,34.22,ORL,32.000000,40.000000,29.538462,31.5,43,11.5,O
3,20251118_BOS_BRK,2025-11-18,BRK,0,Day'Ron Sharpe,C,17.90,BOS,9.076923,10.857143,20.100000,7.5,18,10.5,O
4,20251118_MEM_SAS,2025-11-18,SAS,0,Harrison Barnes,PF,32.15,MEM,15.000000,13.125000,22.000000,15.5,26,10.5,O
5,20251118_GSW_ORL,2025-11-18,ORL,0,Anthony Black,PG,32.75,GSW,14.133333,7.600000,28.111111,13.5,23,9.5,O
6,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,12.000000,8.250000,20.636364,14.5,23,8.5,O
7,20251118_DET_ATL,2025-11-18,ATL,1,Jalen Johnson,SF,39.10,DET,29.714286,26.400000,26.363636,26.5,34,7.5,O
8,20251118_UTA_LAL,2025-11-18,LAL,0,Luka Doncic,PG,34.23,UTA,43.636364,35.777778,29.416667,39.5,47,7.5,O
9,20251118_DET_ATL,2025-11-18,ATL,1,Nickeil Alexander-Walker,SG,36.72,DET,21.714286,14.600000,18.687500,20.5,27,6.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,AVG_RA_H2H,Def_RA,RA_line,RA,RA_Diff,RA_Result
0,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,8.100000,6.250000,8.181818,7.5,18,10.5,O
1,20251118_BOS_BRK,2025-11-18,BOS,0,Payton Pritchard,PG,35.25,BRK,9.333333,7.333333,11.375000,8.5,15,6.5,O
2,20251118_GSW_ORL,2025-11-18,ORL,0,Jalen Suggs,PG,31.93,GSW,8.727273,NaN,13.111111,7.5,13,5.5,O
3,20251118_UTA_LAL,2025-11-18,LAL,0,LeBron James,SF,29.62,UTA,15.000000,15.333333,13.266667,9.5,15,5.5,O
4,20251118_BOS_BRK,2025-11-18,BRK,0,Terance Mann,SG,27.22,BOS,7.076923,9.400000,8.923077,6.5,11,4.5,O
5,20251118_BOS_BRK,2025-11-18,BOS,0,Anfernee Simons,SG,26.15,BRK,4.733333,9.200000,7.866667,4.5,9,4.5,O
6,20251118_UTA_LAL,2025-11-18,LAL,0,Deandre Ayton,C,30.00,UTA,10.461538,8.250000,12.615385,10.5,15,4.5,O
7,20251118_GSW_ORL,2025-11-18,GSW,0,Stephen Curry,PG,34.22,ORL,7.692308,9.400000,12.076923,8.5,12,3.5,O
8,20251118_PHO_POR,2025-11-18,POR,0,Donovan Clingan,C,23.45,PHO,10.428571,7.800000,14.875000,10.5,14,3.5,O
9,20251118_GSW_ORL,2025-11-18,ORL,0,Wendell Carter Jr.,C,35.12,GSW,9.466667,11.400000,14.500000,10.5,14,3.5,O


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,AVG_PRA_H2H,Def_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251118_UTA_LAL,2025-11-18,UTA,0,Keyonte George,PG,35.35,LAL,33.866667,26.875000,27.000000,29.5,46,16.5,O
1,20251118_MEM_SAS,2025-11-18,MEM,0,Vince Williams Jr.,SG,29.83,SAS,16.700000,11.750000,24.545455,18.5,32,13.5,O
2,20251118_GSW_ORL,2025-11-18,GSW,0,Jimmy Butler,SF,38.23,ORL,30.266667,18.857143,29.466667,30.5,44,13.5,O
3,20251118_MEM_SAS,2025-11-18,SAS,0,Harrison Barnes,PF,32.15,MEM,17.928571,16.750000,29.083333,19.5,31,11.5,O
4,20251118_BOS_BRK,2025-11-18,BOS,0,Payton Pritchard,PG,35.25,BRK,26.066667,23.000000,32.875000,25.5,37,11.5,O
5,20251118_UTA_LAL,2025-11-18,LAL,0,Deandre Ayton,C,30.00,UTA,26.214286,19.500000,30.076923,24.5,35,10.5,O
6,20251118_GSW_ORL,2025-11-18,GSW,0,Stephen Curry,PG,34.22,ORL,35.615385,42.800000,34.461538,35.5,46,10.5,O
7,20251118_MEM_SAS,2025-11-18,MEM,0,Cedric Coward,SG,25.50,SAS,23.600000,NaN,24.545455,21.5,31,9.5,O
8,20251118_GSW_ORL,2025-11-18,ORL,0,Anthony Black,PG,32.75,GSW,17.200000,9.200000,33.722222,17.5,27,9.5,O
9,20251118_GSW_ORL,2025-11-18,ORL,0,Wendell Carter Jr.,C,35.12,GSW,22.200000,20.400000,33.700000,22.5,31,8.5,O


In [50]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, AVG_{stat}_H2H, Def_{stat}, {stat}_line, {stat}, {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND MP > 0
                            ORDER BY {stat}_Diff
                            LIMIT 15""").fetchdf())

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,10.166667,1.000000,20.428571,10.5,0,-10.5,U
1,20251118_PHO_POR,2025-11-18,PHO,0,Devin Booker,SG,29.05,POR,27.800000,27.111111,20.166667,29.5,19,-10.5,U
2,20251118_MEM_SAS,2025-11-18,SAS,0,Devin Vassell,SG,34.67,MEM,13.571429,17.285714,18.181818,17.5,10,-7.5,U
3,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,13.181818,12.000000,20.461538,13.5,6,-7.5,U
4,20251118_BOS_BRK,2025-11-18,BRK,0,Terance Mann,SG,27.22,BOS,10.153846,7.600000,13.538462,9.5,3,-6.5,U
5,20251118_PHO_POR,2025-11-18,POR,0,Toumani Camara,PF,32.25,PHO,11.857143,8.444444,18.727273,12.5,6,-6.5,U
6,20251118_PHO_POR,2025-11-18,PHO,0,Dillon Brooks,SF,25.63,POR,20.888889,14.125000,17.666667,17.5,12,-5.5,U
7,20251118_GSW_ORL,2025-11-18,GSW,0,Brandin Podziemski,SG,26.52,ORL,11.500000,7.400000,17.076923,10.5,5,-5.5,U
8,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,7.066667,6.333333,17.076923,7.5,2,-5.5,U
9,20251118_GSW_ORL,2025-11-18,ORL,0,Franz Wagner,SF,36.38,GSW,22.800000,21.000000,15.272727,23.5,18,-5.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,AVG_AST_H2H,Def_AST,AST_line,AST,AST_Diff,AST_Result
0,20251118_UTA_LAL,2025-11-18,LAL,0,Austin Reaves,SG,33.43,UTA,7.583333,5.000000,5.466667,5.5,1,-4.5,U
1,20251118_MEM_SAS,2025-11-18,SAS,0,De'Aaron Fox,PG,33.20,MEM,6.166667,5.142857,5.181818,7.5,3,-4.5,U
2,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,3.777778,2.500000,2.692308,3.5,0,-3.5,U
3,20251118_GSW_ORL,2025-11-18,GSW,0,Brandin Podziemski,SG,26.52,ORL,3.357143,2.400000,3.538462,2.5,0,-2.5,U
4,20251118_MEM_SAS,2025-11-18,MEM,0,Cedric Coward,SG,25.50,SAS,2.866667,NaN,4.272727,3.5,1,-2.5,U
5,20251118_UTA_LAL,2025-11-18,LAL,0,Marcus Smart,SG,17.15,UTA,3.800000,3.166667,5.466667,2.5,1,-1.5,U
6,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,2.400000,0.833333,3.538462,1.5,0,-1.5,U
7,20251118_PHO_POR,2025-11-18,PHO,0,Royce O'Neale,SF,31.17,POR,2.933333,2.285714,3.583333,2.5,1,-1.5,U
8,20251118_PHO_POR,2025-11-18,PHO,0,Jordan Goodwin,PG,21.80,POR,2.300000,2.111111,5.555556,2.5,1,-1.5,U
9,20251118_GSW_ORL,2025-11-18,ORL,0,Franz Wagner,SF,36.38,GSW,3.800000,3.200000,2.090909,4.5,3,-1.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_REB,AVG_REB_H2H,Def_REB,REB_line,REB,REB_Diff,REB_Result
0,20251118_BOS_BRK,2025-11-18,BOS,0,Jordan Walsh,PF,11.22,BRK,5.625000,0.857143,7.444444,5.5,1,-4.5,U
1,20251118_MEM_SAS,2025-11-18,SAS,0,Julian Champagnie,SF,24.38,MEM,4.428571,3.222222,5.411765,4.5,1,-3.5,U
2,20251118_PHO_POR,2025-11-18,POR,0,Toumani Camara,PF,32.25,PHO,5.000000,5.444444,6.181818,5.5,2,-3.5,U
3,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,7.166667,3.000000,11.714286,7.5,4,-3.5,U
4,20251118_DET_ATL,2025-11-18,DET,1,Jalen Duren,C,28.90,ATL,11.923077,6.333333,9.636364,11.5,8,-3.5,U
5,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,6.181818,4.166667,5.307692,5.5,2,-3.5,U
6,20251118_GSW_ORL,2025-11-18,ORL,0,Jonathan Isaac,PF,6.82,GSW,3.545455,5.000000,5.750000,4.5,2,-2.5,U
7,20251118_PHO_POR,2025-11-18,POR,0,Shaedon Sharpe,SG,29.67,PHO,5.000000,1.750000,3.583333,4.5,2,-2.5,U
8,20251118_GSW_ORL,2025-11-18,GSW,0,Will Richard,SG,15.30,ORL,3.200000,NaN,3.769231,2.5,0,-2.5,U
9,20251118_PHO_POR,2025-11-18,PHO,0,Mark Williams,C,21.27,POR,8.153846,NaN,9.181818,8.5,6,-2.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PR,AVG_PR_H2H,Def_PR,PR_line,PR,PR_Diff,PR_Result
0,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,14.857143,4.000000,32.142857,18.5,4,-14.5,U
1,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,19.363636,16.166667,25.769231,19.5,8,-11.5,U
2,20251118_PHO_POR,2025-11-18,POR,0,Toumani Camara,PF,32.25,PHO,16.857143,13.888889,24.909091,18.5,8,-10.5,U
3,20251118_PHO_POR,2025-11-18,PHO,0,Devin Booker,SG,29.05,POR,32.000000,32.111111,24.000000,33.5,25,-8.5,U
4,20251118_MEM_SAS,2025-11-18,SAS,0,Devin Vassell,SG,34.67,MEM,17.071429,20.857143,21.909091,21.5,13,-8.5,U
5,20251118_PHO_POR,2025-11-18,POR,0,Deni Avdija,SF,29.18,PHO,31.928571,22.000000,21.000000,31.5,24,-7.5,U
6,20251118_UTA_LAL,2025-11-18,LAL,0,LeBron James,SF,29.62,UTA,14.000000,25.500000,25.266667,21.5,14,-7.5,U
7,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,8.125000,8.500000,20.846154,9.5,2,-7.5,U
8,20251118_PHO_POR,2025-11-18,PHO,0,Dillon Brooks,SF,25.63,POR,24.000000,18.250000,23.750000,20.5,14,-6.5,U
9,20251118_BOS_BRK,2025-11-18,BOS,0,Jordan Walsh,PF,11.22,BRK,11.875000,1.857143,25.666667,11.5,5,-6.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,AVG_PA_H2H,Def_PA,PA_line,PA,PA_Diff,PA_Result
0,20251118_PHO_POR,2025-11-18,PHO,0,Devin Booker,SG,29.05,POR,34.666667,33.111111,24.000000,36.5,24,-12.5,U
1,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,16.272727,14.500000,23.153846,17.5,6,-11.5,U
2,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,10.714286,1.800000,24.000000,13.5,3,-10.5,U
3,20251118_GSW_ORL,2025-11-18,GSW,0,Brandin Podziemski,SG,26.52,ORL,14.437500,9.800000,20.615385,14.5,5,-9.5,U
4,20251118_PHO_POR,2025-11-18,POR,0,Toumani Camara,PF,32.25,PHO,14.785714,9.444444,22.090909,16.5,9,-7.5,U
5,20251118_PHO_POR,2025-11-18,POR,0,Deni Avdija,SF,29.18,PHO,30.285714,19.285714,19.100000,31.5,24,-7.5,U
6,20251118_GSW_ORL,2025-11-18,ORL,0,Franz Wagner,SF,36.38,GSW,26.600000,24.200000,17.363636,28.5,21,-7.5,U
7,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,8.125000,7.166667,20.615385,8.5,2,-6.5,U
8,20251118_MEM_SAS,2025-11-18,SAS,0,Devin Vassell,SG,34.67,MEM,16.142857,21.000000,21.636364,21.5,15,-6.5,U
9,20251118_GSW_ORL,2025-11-18,GSW,0,Moses Moody,SG,29.57,ORL,15.230769,10.000000,20.615385,13.5,8,-5.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,AVG_RA_H2H,Def_RA,RA_line,RA,RA_Diff,RA_Result
0,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,9.272727,6.666667,8.000000,9.5,2,-7.5,U
1,20251118_MEM_SAS,2025-11-18,SAS,0,De'Aaron Fox,PG,33.20,MEM,9.333333,9.285714,10.272727,11.5,4,-7.5,U
2,20251118_BOS_BRK,2025-11-18,BOS,0,Jordan Walsh,PF,11.22,BRK,6.875000,1.000000,12.555556,7.5,1,-6.5,U
3,20251118_BOS_BRK,2025-11-18,BRK,0,Noah Clowney,PF,22.68,BOS,5.357143,1.571429,8.700000,6.5,1,-5.5,U
4,20251118_UTA_LAL,2025-11-18,LAL,0,Austin Reaves,SG,33.43,UTA,12.666667,9.571429,9.600000,10.5,6,-4.5,U
5,20251118_PHO_POR,2025-11-18,POR,0,Robert Williams,C,8.10,PHO,4.333333,1.200000,14.875000,6.5,2,-4.5,U
6,20251118_MEM_SAS,2025-11-18,SAS,0,Julian Champagnie,SF,24.38,MEM,5.571429,4.444444,9.411765,5.5,1,-4.5,U
7,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,3.428571,3.000000,7.307692,3.5,0,-3.5,U
8,20251118_PHO_POR,2025-11-18,POR,0,Deni Avdija,SF,29.18,PHO,11.214286,8.714286,8.100000,13.5,10,-3.5,U
9,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,8.142857,3.800000,15.285714,10.5,7,-3.5,U


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,AVG_PRA_H2H,Def_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251118_MEM_SAS,2025-11-18,SAS,0,Luke Kornet,C,25.08,MEM,16.857143,4.800000,35.714286,21.5,7,-14.5,U
1,20251118_DET_ATL,2025-11-18,DET,1,Ausar Thompson,SF,23.75,ATL,22.454545,18.666667,28.461538,22.5,8,-14.5,U
2,20251118_PHO_POR,2025-11-18,PHO,0,Devin Booker,SG,29.05,POR,38.866667,38.111111,27.833333,40.5,30,-10.5,U
3,20251118_PHO_POR,2025-11-18,POR,0,Toumani Camara,PF,32.25,PHO,19.785714,14.888889,28.272727,21.5,11,-10.5,U
4,20251118_GSW_ORL,2025-11-18,GSW,0,Buddy Hield,SG,15.20,ORL,9.625000,9.333333,24.384615,11.5,2,-9.5,U
5,20251118_PHO_POR,2025-11-18,POR,0,Deni Avdija,SF,29.18,PHO,36.714286,25.000000,24.100000,38.5,29,-9.5,U
6,20251118_BOS_BRK,2025-11-18,BOS,0,Jordan Walsh,PF,11.22,BRK,13.125000,2.000000,30.777778,13.5,5,-8.5,U
7,20251118_MEM_SAS,2025-11-18,SAS,0,Julian Champagnie,SF,24.38,MEM,15.285714,13.000000,26.529412,18.5,10,-8.5,U
8,20251118_GSW_ORL,2025-11-18,GSW,0,Brandin Podziemski,SG,26.52,ORL,19.062500,15.400000,24.384615,18.5,11,-7.5,U
9,20251118_GSW_ORL,2025-11-18,ORL,0,Jonathan Isaac,PF,6.82,GSW,6.727273,10.250000,25.583333,9.5,2,-7.5,U
